In [1]:
%load_ext autoreload
%autoreload 2
from dimcat import (
    Corpus,  
    Pipeline,
    IsAnnotatedFilter,
    CorpusGrouper, 
    PieceGrouper, 
    ModeGrouper, 
    ChordSymbolBigrams, 
    ChordSymbolUnigrams,
    LocalKeySlicer,
)
import pandas as pd
pd.set_option("display.max_columns", 100)

In [2]:
corpus_path = "~/romantic_piano_corpus"
corpus = Corpus(directory=corpus_path)
corpus.data

1154 files.
KEY                       -> EXTENSIONS
---------------------------------------
beethoven_piano_sonatas   -> {'.mscx': 87, '.tsv': 239}
chopin_mazurkas           -> {'.mscx': 55, '.tsv': 166}
debussy_suite_bergamasque -> {'.mscx': 4, '.tsv': 13}
dvorak_silhouettes        -> {'.mscx': 12, '.tsv': 37}
grieg_lyrical_pieces      -> {'.mscx': 66, '.tsv': 199}
liszt_pelerinage          -> {'.mscx': 19, '.tsv': 77}
medtner_tales             -> {'.mscx': 19, '.tsv': 59}
schumann_kinderszenen     -> {'.mscx': 13, '.tsv': 40}
tchaikovsky_seasons       -> {'.mscx': 12, '.tsv': 37}

None of the 287 score files have been parsed.

All 867 tabular files have been parsed, 283 of them as Annotations object(s).
KEY                       -> ANNOTATION LAYERS
----------------------------------------------
beethoven_piano_sonatas   -> staff  voice  harmony_layer   color  
                          -> 2      1      0 (Plain Text)  default    20720
                          ->        2      0 (Pl

# Metadata

In [3]:
all_metadata = corpus.data.metadata(from_tsv=True)
print(f"Concatenated 'metadata.tsv' files cover {len(all_metadata)} of the {len(corpus.data._score_ids())} scores.")
all_metadata.sample(10)

Concatenated 'metadata.tsv' files cover 287 of the 287 scores.


rel_paths  \
key                     i   metadata_id             
schumann_kinderszenen   39  6                 MS3   
medtner_tales           58  16                MS3   
beethoven_piano_sonatas 238 63                MS3   
                            59                MS3   
grieg_lyrical_pieces    198 1                 MS3   
                            26                MS3   
dvorak_silhouettes      36  9                 MS3   
beethoven_piano_sonatas 238 30                MS3   
liszt_pelerinage        76  14                MS3   
beethoven_piano_sonatas 238 85                MS3   

                                                                  fnames  \
key                     i   metadata_id                                    
schumann_kinderszenen   39  6                                        n07   
medtner_tales           58  16                                   op42n03   
beethoven_piano_sonatas 238 63                                      18-4   
                            59                                      17-3   
grieg_lyrical_pieces    198 1                                    op12n02   
                            26                                   op47n05   
dvorak_silhouettes      36  9                                    op08n10   
beethoven_piano_sonatas 238 30                                      09-2   
liszt_pelerinage        76  14           161.06_Sonetto_123_del_Petrarca   
beethoven_piano_sonatas 238 85                                      32-1   

                                         last_mc  last_mn  length_qb  \
key                     i   metadata_id                                
schumann_kinderszenen   39  6                 25       24      97.00   
medtner_tales           58  16               183      182        NaN   
beethoven_piano_sonatas 238 63               336      333    1005.00   
                            59               400      399     598.50   
grieg_lyrical_pieces    198 1                 79       79        NaN   
                            26                41       41        NaN   
dvorak_silhouettes      36  9                 58       58        NaN   
beethoven_piano_sonatas 238 30               178      178     534.00   
liszt_pelerinage        76  14                84       84     345.00   
beethoven_piano_sonatas 238 85               160      157     636.12   

                                         length_qb_unfolded  all_notes_qb  \
key                     i   metadata_id                                     
schumann_kinderszenen   39  6                        129.00        428.50   
medtner_tales           58  16                          NaN           NaN   
beethoven_piano_sonatas 238 63                      1236.00       1995.00   
                            59                       739.50       1468.13   
grieg_lyrical_pieces    198 1                           NaN           NaN   
                            26                          NaN           NaN   
dvorak_silhouettes      36  9                           NaN           NaN   
beethoven_piano_sonatas 238 30                       582.00       1959.50   
liszt_pelerinage        76  14                       345.00        936.17   
beethoven_piano_sonatas 238 85                       836.12       1970.58   

                                         n_onsets  n_onset_positions  \
key                     i   metadata_id                                
schumann_kinderszenen   39  6               360.0              162.0   
medtner_tales           58  16                NaN                NaN   
beethoven_piano_sonatas 238 63             2769.0                NaN   
                            59             3611.0                NaN   
grieg_lyrical_pieces    198 1                 NaN                NaN   
                            26                NaN                NaN   
dvorak_silhouettes      36  9                 NaN                NaN   
beethoven_piano_sonatas 238 30             1215.0   

In [4]:
print("VALUE COUNTS OF THE COLUMN 'annotators'")
all_metadata.annotators.value_counts()

VALUE COUNTS OF THE COLUMN 'annotators'


Adrian Nagel (2.1.1), John Heilig (2.3.0)                               74
Wendelin Bitzan (1.0.0), Adrian Nagel (2.2.0), Davor Krkljus (2.3.0)    49
Adrian Nagel (2.3.0)                                                    16
Adrian Nagel (2.1.1), Amelia Brey (2.3.0)                               15
Wendelin Bitzan (2.2.0), John Heilig (2.3.0)                            15
Tal Soker (2.1.1), John Heilig (2.3.0)                                  13
Lydia Carlisi (2.2.0), Adrian Nagel (2.3.0)                             11
Adrian Nagel                                                             9
Adrian Nagel (2.1.1), Hanné Becker (2.3.0)                               9
Adrian Nagel (2.2.0), Amelia Brey (2.3.0)                                6
Daniel Grote (2.2.0), Adrian Nagel (2.3.0)                               6
Adrian Nagel (2.2.0), Hanné Becker (2.3.0)                               5
Wendelin Bitzan                                                          5
Lydia Carlisi (2.2.0), Vi

In [5]:
print(f"Composition dates range from {all_metadata.composed_start.min()} ({all_metadata.loc[all_metadata.composed_start.idxmin(), 'fnames']}) "
      f"to {all_metadata.composed_end.max()} ({all_metadata.loc[all_metadata.composed_end.idxmax(), 'fnames']}).")

Composition dates range from 1794 (01-1) to 1925 (op08n01).


In [6]:
annotated = IsAnnotatedFilter().process_data(corpus)
print(f"Before: {len(corpus.indices[()])} IDs, after filtering: {len(annotated.indices[()])}")

Before: 287 IDs, after filtering: 264


**Choose here if you want to see stats for all or only for annotated scores.**

In [7]:
#selected = corpus
selected = annotated

# Notes

In [8]:
all_notes = selected.get_facet('notes')
all_notes.sample(20)

quarterbeats  \
corpus                    fname                               interval                                                
grieg_lyrical_pieces      op62n02                             [292.5, 293.0)                                  585/2   
                          op65n06                             [471.0, 471.25)                                   471   
liszt_pelerinage          161.07_Apres_une_lecture_du_Dante   [1366.5833333333333, 1366.9166666666665)     16399/12   
beethoven_piano_sonatas   32-2                                [285.25, 285.5)                                1141/4   
liszt_pelerinage          161.03_Canzonetta_del_Salvator_Rosa [58.0, 59.0)                                       58   
                          161.07_Apres_une_lecture_du_Dante   [972.0, 973.0)                                    972   
schumann_kinderszenen     n08                                 [17.5, 18.0)                                     35/2   
liszt_pelerinage          160.07_Eglogue                      [360.5, 361.0)                                  721/2   
                          161.02_Il_Pensieroso                [19.75, 20.0)                                    79/4   
chopin_mazurkas           BI115-2op33-2                       [383.5, 383.5)                                  767/2   
tchaikovsky_seasons       op37a02                             [59.0, 59.5)                                       59   
grieg_lyrical_pieces      op68n04                             [97.0, 97.0)                                       97   
medtner_tales             op34n03                             [17.0, 17.25)                                      17   
beethoven_piano_sonatas   31-1                                [214.25, 214.375)                               857/4   
                          09-3                                [126.0, 127.5)                                    126   
liszt_pelerinage          161.01_Sposalizio                   [639.0, 640.0)                                    639   
beethoven_piano_sonatas   23-1                                [1380.75, 1381.0)                              5523/4   
debussy_suite_bergamasque l075-04_suite_passepied             [20.0, 24.0)                                       20   
beethoven_piano_sonatas   26-3                                [494.5, 495.0)                                  989/2   
chopin_mazurkas           BI157-1op59-1                       [334.0, 335.0)                                    334   

                                                                                                        duration_qb  \
corpus                    fname                               interval                                                
grieg_lyrical_pieces      op62n02                             [292.5, 293.0)                               0.500000   
                          op65n06                             [471.0, 471.25)                              0.250000   
liszt_pelerinage          161.07_Apres_une_lecture_du_Dante   [1366.5833333333333, 1366.9166666666665)     0.333333   
beethoven_piano_sonatas   32-2                                [285.25, 285.5)                              0.250000   
liszt_pelerinage          161.03_Canzonetta_del_Salvator_Rosa [58.0, 59.0)                                 1.000000   
                          161.07_Apres_une_lecture_du_Dante   [972.0, 973.0)                               1.000000   
schumann_kinderszenen     n08                                 [17.5, 18.0)                                 0.500000   
liszt_pelerinage          160.07_Eglogue                      [360.5, 361.0)                               0.500000   
                          161.02_Il_Pensieroso                [19.75, 20.0)                                0.250000   
chopin_mazurkas           BI115-2op33-2                       [383.5, 383.5)                               0.000000   
tchaikovsky_seasons       op37a02                       

In [9]:
print(f"{len(all_notes)} notes distributed over staves as follows:")
all_notes.staff.value_counts()

434220 notes distributed over staves as follows:


1    230221
2    200618
3      2397
4       984
Name: staff, dtype: Int64

In [10]:
print("Distribution of notes over staves for all pieces with more than two staves\n")
for group, df in all_notes.groupby(level=[0,1]):
    if (df.staff > 2).any():
        print(group)
        print(df.staff.value_counts().to_dict())

Distribution of notes over staves for all pieces with more than two staves

('grieg_lyrical_pieces', 'op43n06')
{2: 769, 3: 422, 1: 180}
('liszt_pelerinage', '161.04_Sonetto_47_del_Petrarca')
{1: 1076, 2: 628, 3: 42, 4: 29}
('liszt_pelerinage', '161.07_Apres_une_lecture_du_Dante')
{1: 6638, 2: 5181, 3: 50}
('liszt_pelerinage', '162.01_Gondoliera')
{3: 1745, 4: 955}
('medtner_tales', 'op34n03')
{1: 1219, 2: 816, 3: 89}
('medtner_tales', 'op35n04')
{1: 1678, 2: 1632, 3: 49}


In [11]:
all_notes[all_notes.staff > 2].groupby(level=[0,1]).staff.value_counts()

corpus                fname                              staff
grieg_lyrical_pieces  op43n06                            3         422
liszt_pelerinage      161.04_Sonetto_47_del_Petrarca     3          42
                                                         4          29
                      161.07_Apres_une_lecture_du_Dante  3          50
                      162.01_Gondoliera                  3        1745
                                                         4         955
medtner_tales         op34n03                            3          89
                      op35n04                            3          49
Name: staff, dtype: int64

# Tokens

All symbols, independent of the local key (the mode of which changes their semantics).

In [12]:
for group, dict_of_dfs in annotated.iter_facet("expanded", concatenate=True):
    for tuple_of_all_id_tuples, all_annotations in dict_of_dfs.items():
        pass
all_annotations.sample(20)

quarterbeats  \
corpus                  fname                        interval                      
chopin_mazurkas         BI157-3op59-3                [354.0, 355.0)          354   
liszt_pelerinage        160.06_Vallee_dObermann      [527.5, 529.5)       1055/2   
grieg_lyrical_pieces    op54n03                      [144.0, 146.0)          144   
schumann_kinderszenen   n10                          [8.25, 9.25)           33/4   
grieg_lyrical_pieces    op54n03                      [376.5, 377.5)        753/2   
beethoven_piano_sonatas 18-1                         [318.0, 319.0)          318   
chopin_mazurkas         BI162-2op63-2                [86.0, 87.0)             86   
liszt_pelerinage        160.05_Orage                 [216.5, 217.0)        433/2   
                        160.02_Au_Lac_de_Wallenstadt [96.0, 99.0)             96   
medtner_tales           op48n02                      [184.0, 185.5)          184   
liszt_pelerinage        161.01_Sposalizio            [287.0, 288.0)          287   
beethoven_piano_sonatas 23-1                         [734.5, 737.5)       1469/2   
grieg_lyrical_pieces    op47n01                      [86.0, 87.0)             86   
beethoven_piano_sonatas 08-1                         [495.0, 496.0)          495   
                        06-1                         [252.5, 253.5)        505/2   
                        30-2                         [402.0, 403.0)          402   
                        18-2                         [112.0, 113.5)          112   
                        19-2                         [95.0, 98.0)             95   
grieg_lyrical_pieces    op62n06                      [254.0, 256.0)          254   
beethoven_piano_sonatas 02-1                         [508.0, 508.5)          508   

                                                                    duration_qb  \
corpus                  fname                        interval                     
chopin_mazurkas         BI157-3op59-3                [354.0, 355.0)         1.0   
liszt_pelerinage        160.06_Vallee_dObermann      [527.5, 529.5)         2.0   
grieg_lyrical_pieces    op54n03                      [144.0, 146.0)         2.0   
schumann_kinderszenen   n10                          [8.25, 9.25)           1.0   
grieg_lyrical_pieces    op54n03                      [376.5, 377.5)         1.0   
beethoven_piano_sonatas 18-1                         [318.0, 319.0)         1.0   
chopin_mazurkas         BI162-2op63-2                [86.0, 87.0)           1.0   
liszt_pelerinage        160.05_Orage                 [216.5, 217.0)         0.5   
                        160.02_Au_Lac_de_Wallenstadt [96.0, 99.0)           3.0   
medtner_tales           op48n02                      [184.0, 185.5)         1.5   
liszt_pelerinage        161.01_Sposalizio            [287.0, 288.0)         1.0   
beethoven_piano_sonatas 23-1                         [734.5, 737.5)         3.0   
grieg_lyrical_pieces    op47n01                      [86.0, 87.0)           1.0   
beethoven_piano_sonatas 08-1                         [495.0, 496.0)         1.0   
                        06-1                         [252.5, 253.5)         1.0   
                        30-2                         [402.0, 403.0)         1.0   
                        18-2                         [112.0, 113.5)         1.5   
                        19-2                         [95.0, 98.0)           3.0   
grieg_lyrical_pieces    op62n06                      [254.0, 256.0)         2.0   
beethoven_piano_sonatas 02-1                         [508.0, 508.5)         0.5   

                                                                      mc   mn  \
corpus                  fname                        interval                   
chopin_mazurkas         BI157-3op59-3                [354.0, 355.0)  119  118   
liszt_pelerinage        160.06_Vallee_dObermann      [527.5, 529.5)  135  131   
grieg_lyrical_pieces    op54n03                      [144.0, 146.0) 

In [13]:
no_chord = all_annotations.root.isna()
print(f"Concatenated annotation tables contains {all_annotations.shape[0]} rows. {no_chord.sum()} of them are not chords. Their values are:")
all_annotations.label[no_chord].value_counts(dropna=False).to_dict()

Concatenated annotation tables contains 57566 rows. 677 of them are not chords. Their values are:


{'{': 640, '}': 28, '|PAC}': 6, '|HC': 2, '|PAC': 1}

In [14]:
all_chords = all_annotations[~no_chord]
print(f"Corpus contains {all_chords.shape[0]} tokens and {len(all_chords.chord.unique())} types over {len(tuple_of_all_id_tuples)} documents.")

Corpus contains 56889 tokens and 3115 types over 264 documents.


In [15]:
#from ms3 import write_tsv
#write_tsv(all_annotations[all_annotations.pedalend.notna()], './issues/pedalpoints.tsv', pre_process=False)

# Unigrams
For computing unigram statistics, the tokens need to be grouped by their occurrence within a major or a minor key because this changes their meaning. To that aim, the annotated corpus needs to be sliced into contiguous localkey segments which are then grouped into a major (`is_minor=False`) and a minor group.

In [16]:
localkey_slices = LocalKeySlicer().process_data(annotated)

In [17]:
mode_slices = ModeGrouper().process_data(localkey_slices)

## Whole dataset

In [18]:
unigrams = ChordSymbolUnigrams(once_per_group=True).process_data(mode_slices)

In [19]:
unigrams.get()

localkey_is_minor  chord        
False              I                5229
                   V7               2597
                   V                2176
                   I6               1757
                   IV                981
                                    ... 
True               I(#6)               1
                   V(642)              1
                   I(94)               1
                   It/iv               1
                   #viio2(+2)/ii       1
Name: count, Length: 3715, dtype: int64

In [20]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in unigrams.iter():
    print(f"{modes[is_minor]} UNIGRAMS\n{ugs.shape[0]} types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR UNIGRAMS
2004 types, 35610 tokens
chord
I        5229
V7       2597
V        2176
I6       1757
IV        981
V(64)     722
V43       670
V2        647
V6        638
vi        610
ii        570
ii6       548
V65       541
IV6       469
I64       445
V7(9)     363
viio6     347
V7/V      324
ii7       278
V7/IV     236
MINOR UNIGRAMS
1711 types, 21279 tokens
chord
i          2703
V          1413
V7         1173
i6          914
iv          489
V(64)       458
V6          354
VI          321
iv6         314
V43         303
i64         243
III         233
V65         222
V2          213
#viio43     209
I           208
v           170
bII         161
V7/III      158
#viio7      156


## Per corpus

In [21]:
corpus_wise_unigrams = Pipeline([CorpusGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [22]:
corpus_wise_unigrams.get()

localkey_is_minor  corpus                   chord   
False              beethoven_piano_sonatas  I           2232
                                            V           1189
                                            I6          1125
                                            V7          1041
                                            V6           468
                                                        ... 
True               tchaikovsky_seasons      i(4)/v         1
                                            i/v            1
                                            ii64           1
                                            v6/iv          1
                                            ii%65(2)       1
Name: count, Length: 6705, dtype: int64

In [23]:
for (is_minor, corpus_name), ugs in corpus_wise_unigrams.iter():
    print(f"{corpus_name} {modes[is_minor]} unigrams ({ugs.shape[0]} types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

beethoven_piano_sonatas MAJOR unigrams (842 types, 15275 tokens)
chord
I     2232
V     1189
I6    1125
V7    1041
V6     468
chopin_mazurkas MAJOR unigrams (451 types, 5256 tokens)
chord
I       949
V7      659
V       252
IV      189
V7/V    128
debussy_suite_bergamasque MAJOR unigrams (106 types, 363 tokens)
chord
I         28
V7        22
iii       19
V7(+2)    12
IV6       12
dvorak_silhouettes MAJOR unigrams (140 types, 1067 tokens)
chord
I     254
I6    103
V7     89
V      79
IV     46
grieg_lyrical_pieces MAJOR unigrams (659 types, 4882 tokens)
chord
I     566
V7    264
V     224
IV    127
vi    116
liszt_pelerinage MAJOR unigrams (552 types, 3455 tokens)
chord
I     501
V7    250
V     155
I6    109
IV    108
medtner_tales MAJOR unigrams (636 types, 2864 tokens)
chord
I     263
V     139
V7    122
I6     83
IV     61
schumann_kinderszenen MAJOR unigrams (114 types, 714 tokens)
chord
I      106
I6      69
V       65
V7      50
V43     30
tchaikovsky_seasons MAJOR unigrams (194

## Per piece

In [24]:
piece_wise_unigrams = Pipeline([PieceGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [25]:
piece_wise_unigrams.get()

localkey_is_minor  fname    chord 
False              01-1     V7        11
                            I6         8
                            I          7
                            ii6        5
                            V          5
                                      ..
True               op71n07  ii%65      3
                            VIM43      2
                            ii%7       2
                            V(964)     2
                            #viio2     1
Name: count, Length: 16090, dtype: int64

# Bigrams

## Whole dataset

In [26]:
bigrams = ChordSymbolBigrams(once_per_group=True).process_data(mode_slices)

In [27]:
bigrams.get()

localkey_is_minor             to      
False              group_ids  I           1254
                              I            586
                              V            443
                              V7           406
                              I6           344
                                          ... 
True               group_ids  bVI            1
                              VIM7(+2)       1
                              VIM7           1
                              VI(11)         1
                              i              1
Name: count, Length: 15149, dtype: int64

In [28]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in bigrams.iter():
    print(f"{modes[is_minor]} BIGRAMS\n{ugs.shape[0]} transition types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR BIGRAMS
8539 transition types, 34721 tokens
           to 
group_ids  I      1254
           I       586
           V       443
           V7      406
           I6      344
           V7      310
           I6      294
           I       292
           I       266
           IV      245
           V7      233
           I       218
           V43     210
           I       164
           I       164
           V6      161
           I6      147
           V       142
           IV      139
           V2      132
MINOR BIGRAMS
6610 transition types, 20690 tokens
           to   
group_ids  i        573
           i        417
           V        209
           V7       192
           i6       172
           V7       149
           i        145
           V7       142
           V        138
           i        133
           i        129
           i        118
           V6       104
           i6        94
           iv        89
           VI        80
           V43       75


## Per corpus

In [29]:
corpus_wise_bigrams = Pipeline([CorpusGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

In [30]:
corpus_wise_bigrams.get()

localkey_is_minor  corpus                              to      
False              beethoven_piano_sonatas  group_ids  I           532
                                                       I           323
                                                       I6          238
                                                       V           230
                                                       V7          213
                                                                  ... 
True               tchaikovsky_seasons      group_ids  i/v           1
                                                       ii%65(2)      1
                                                       V7            1
                                                       V             1
                                                       ii%7(4)       1
Name: count, Length: 19367, dtype: int64

In [31]:
for (is_minor, corpus_name), ugs in corpus_wise_bigrams.iter():
    print(f"{corpus_name} {modes[is_minor]} bigrams ({ugs.shape[0]} transition types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

beethoven_piano_sonatas MAJOR bigrams (3509 transition types, 15028 tokens)
           to
group_ids  I     532
           I     323
           I6    238
           V     230
           V7    213
chopin_mazurkas MAJOR bigrams (1394 transition types, 5135 tokens)
           to
group_ids  I     338
           IV     96
           V7     93
           V      70
           I      67
debussy_suite_bergamasque MAJOR bigrams (245 transition types, 354 tokens)
           to   
group_ids  iii      8
           I        7
           V7       5
           bIII6    5
           V64      4
dvorak_silhouettes MAJOR bigrams (347 transition types, 1013 tokens)
           to
group_ids  I     64
           V7    45
           I     37
           I6    29
           I6    21
grieg_lyrical_pieces MAJOR bigrams (1735 transition types, 4755 tokens)
           to
group_ids  I     102
           V      39
           I      37
           I      31
           V7     30
liszt_pelerinage MAJOR bigrams (1471 transi

## Per piece

In [32]:
piece_wise_bigrams = Pipeline([PieceGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

(('beethoven_piano_sonatas', '23-2', Interval(190.0, 194.0, closed='left')),): DataFrame has only one row, cannot compute bigram.
Group '(True, '23-2')' will be missing from the processed data.


In [33]:
piece_wise_bigrams.get()

localkey_is_minor  fname               to    
False              01-1     group_ids  V7        4
                                       V(64)     3
                                       ii6       3
                                       V         3
                                       V7        3
                                                ..
True               op71n07  group_ids  ii%65     2
                                       ii%7      2
                                       VIM43     2
                                       V(964)    2
                                       #viio2    1
Name: count, Length: 27544, dtype: int64